In [1]:
import numpy as np
import cv2
from pyproj import Transformer
from my_vbr_utils.vbr_dataset import vbrInterpolatedDataset, get_paths_from_scene, load_calibration
from my_vbr_utils.utilities import load_scene_correspondences


location = 'ciampino_train0'
dataset_root = '/datasets/vbr_slam'

# Load dataset and calibration
vbr_scene = vbrInterpolatedDataset(dataset_root, location)
calib_path=get_paths_from_scene(dataset_root, location)[-1]
calib = load_calibration(calib_path)


Loaded vbrInterpolatedDataset from scene:  ciampino_train0
Images loaded from:  /datasets/vbr_slam/ciampino/ciampino_train0_kitti/camera_left/data
Ground Truth Poses:  /datasets/vbr_slam/ciampino/ciampino_train0/ciampino_train0_gt.txt


In [2]:
# vbr_scene_traj = vbr_scene.get_local_trajectory()

In [3]:
# import matplotlib.pyplot as plt
# import numpy as np
# n = 2000 #step 

# x1, y1 = vbr_scene_traj[:, 0], vbr_scene_traj[:, 1]
# indices = np.arange(0, len(x1), n)
# plt.figure(figsize=(8, 5))
# plt.plot(x1, y1, 'b-', alpha=0.5, label='')

# plt.scatter(x1[0], y1[0], color='red', marker='x', s=60, label='Start')
# plt.scatter(x1[-1], y1[-1], color='green', marker='x', s=60, label='End')
# plt.scatter(x1[indices[1:-1]], y1[indices[1:-1]], color='blue', marker='o', s=60, label='')

# # Annotate each anchor with its index (excluding first and last)
# for idx in indices[1:-1]:
#     plt.text(x1[idx] + 2, y1[idx] + 2, str(idx), color='blue', fontsize=8,  # <-- increased fontsize here
#              ha='left', va='bottom',
#              bbox=dict(facecolor='white', edgecolor='none', alpha=0.7, pad=0.5))

# plt.xlabel('X (m)', fontsize=14)
# plt.ylabel('Y (m)', fontsize=14)
# plt.legend(fontsize=12)
# plt.grid(True, linestyle='--', alpha=0.5)
# plt.title(f"{location} in Local Frame", fontsize=16)
# plt.tight_layout()
# plt.show()



In [4]:
import matplotlib.pyplot as plt
import numpy as np
import json
json_path = f"/home/bjangley/VPR/mast3r-v2/my_vbr_utils/vbr_sequences/{location}.json"
def load_anchor_query_dict(json_file_path):
    """
    Load the anchor-query dictionary from a JSON file.
    """
    with open(json_file_path, 'r') as f:
        loaded_dict = json.load(f)

    # Convert string keys back to tuples
    anchor_query_dict = {tuple(map(int, key.strip("()").split(","))): value for key, value in loaded_dict.items()}
    return anchor_query_dict
anchor_query_dict=load_anchor_query_dict(json_path)
print(anchor_query_dict)


{(0, 1145): [[3026, 3421]], (1145, 1496): [[3421, 3772], [16708, 17244]], (1496, 2456): [[29980, 30849]], (3773, 4594): [[17246, 18245]], (4600, 5390): [[6000, 7300], [18300, 19000]], (13236, 14000): [[11470, 12115]], (14396, 15046): [[24900, 26000]], (15113, 16612): [[21360, 21900], [26986, 27735]]}


In [5]:
# import matplotlib.pyplot as plt
# import numpy as np
# import ipywidgets as widgets


# def plot_subplots_with_start_stop(anchor_range):
#     query_ranges = anchor_query_dict.get(anchor_range)
#     if query_ranges is None:
#         fig, ax = plt.subplots(figsize=(8, 4))
#         ax.text(0.5, 0.5, f"No query ranges found for anchor range {anchor_range}.",
#                 ha='center', va='center')
#         ax.axis('off')
#         return fig

#     n_total = 1 + len(query_ranges)
#     n_cols = 3
#     n_rows = int(np.ceil(n_total / n_cols))
#     fig, axs = plt.subplots(n_rows, n_cols, figsize=(7 * n_cols, 5 * n_rows), squeeze=False)
#     axs = axs.flatten()

#     seg = vbr_scene_traj
#     n = 50
#     # 1) Plot the anchor
#     anchor_seq = vbr_scene_traj[anchor_range[0]:anchor_range[1]]
#     axs[0].plot(seg[:,0], seg[:,1], color='red', alpha=0.3, label="Full Trajectory")
#     axs[0].plot(anchor_seq[:,0], anchor_seq[:,1], 'g-', linewidth=2.5, label=f"Anchor {anchor_range}")
#     # Start marker
#     axs[0].plot(anchor_seq[0,0], anchor_seq[0,1], 'o', color='green', markersize=12, label='Start')
#     # axs[0].text(anchor_seq[0,0], anchor_seq[0,1]-n, f"Start: {anchor_range[0]}", fontsize=10,
#     #             color='green', va='bottom', ha='right', bbox=dict(facecolor='white', alpha=0.7))
#     # End marker
#     axs[0].plot(anchor_seq[-1,0], anchor_seq[-1,1], 'X', color='darkgreen', markersize=12, label='End')
#     # axs[0].text(anchor_seq[-1,0], anchor_seq[-1,1]-n, f"End: {anchor_range[1]}", fontsize=10,
#     #             color='darkgreen', va='top', ha='left', bbox=dict(facecolor='white', alpha=0.7))
#     axs[0].set_title(f"Anchor {anchor_range}")
#     axs[0].legend()
#     axs[0].grid(True, linestyle='--')

#     # 2) Plot each query range
#     for i, (qs, qe) in enumerate(query_ranges, start=1):
#         query_seq = vbr_scene_traj[qs:qe]
#         axs[i].plot(seg[:,0], seg[:,1], color='red', alpha=0.3, label="Full Trajectory")
#         axs[i].plot(query_seq[:,0], query_seq[:,1], 'blue', linewidth=2, label=f"Query {i}: {qs}-{qe}")
#         # Start marker
#         axs[i].plot(query_seq[0,0], query_seq[0,1], 'o', color='blue', markersize=12, label='Start')
#         # axs[i].text(query_seq[0,0], query_seq[0,1]-n, f"Start: {qs}", fontsize=10,
#         #             color='purple', va='bottom', ha='right', bbox=dict(facecolor='white', alpha=0.7))
#         # End marker
#         axs[i].plot(query_seq[-1,0], query_seq[-1,1], 'X', color='blue', markersize=12, label='End')
#         # axs[i].text(query_seq[-1,0], query_seq[-1,1]-n, f"End: {qe}", fontsize=10,
#         #             color='indigo', va='top', ha='left', bbox=dict(facecolor='white', alpha=0.7))
#         axs[i].set_title(f"Query {i}: {qs}-{qe}")
#         axs[i].legend()
#         axs[i].grid(True, linestyle='--')

#     # Hide unused subplots
#     for i in range(n_total, len(axs)):
#         axs[i].axis('off')

#     fig.tight_layout()
#     return fig


# anchor_ranges = list(anchor_query_dict.keys())
# range_names = [str(r) for r in anchor_ranges]
# range_selector = widgets.SelectionSlider(
#     options=range_names,
#     description='Anchor:',
#     continuous_update=False
# )

# def update_plot(anchor_range_str):
#     anchor_range = eval(anchor_range_str)
#     plt.close('all')
#     return plot_subplots_with_start_stop(anchor_range)

# widget = widgets.interactive(update_plot, anchor_range_str=range_selector)
# display(widget)


In [6]:
import json
json_path = "/home/bjangley/VPR/mast3r-v2/my_vbr_utils/vbr_sequences/ciampino_train0.json"
def load_anchor_query_dict(json_file_path):
    """
    Load the anchor-query dictionary from a JSON file.
    """
    with open(json_file_path, 'r') as f:
        loaded_dict = json.load(f)

    # Convert string keys back to tuples
    anchor_query_dict = {tuple(map(int, key.strip("()").split(","))): value for key, value in loaded_dict.items()}
    return anchor_query_dict
def calculate_total_queries(anchor_query_dict, step_size):
    """
    Calculates the total number of query sequences after subsampling with a given step size.

    Args:
        anchor_query_dict (dict): A dictionary where keys are anchor ranges (tuples) and
                                   values are lists of query ranges (lists of tuples).
        step_size (int): The step size for subsampling the query ranges.

    Returns:
        int: The total number of query sequences after subsampling.
    """
    total_queries = 0
    for anchor_range, query_ranges in anchor_query_dict.items():
        for query_range in query_ranges:
            start, end = query_range
            # Calculate the number of subsampled queries for this range
            num_queries = (end - start + step_size - 1) // step_size  # Ceiling division
            total_queries += num_queries
    return total_queries

def calculate_total_anchors(anchor_query_dict, anchor_step_size):
    """
    Calculates the total number of anchor sequences after subsampling with a given step size.

    Args:
        anchor_query_dict (dict): A dictionary where keys are anchor ranges (tuples) and
                                   values are lists of query ranges (lists of tuples).
        anchor_step_size (int): The step size for subsampling the anchor ranges.

    Returns:
        int: The total number of anchor sequences after subsampling.
    """
    total_anchors = 0
    for anchor_range, query_ranges in anchor_query_dict.items():
        start, end = anchor_range
        num_anchors = (end - start + anchor_step_size - 1) // anchor_step_size
        total_anchors += num_anchors
    return total_anchors

# Example usage:
step_size =20  
anchor_step_size = 10
anchor_query_dict=load_anchor_query_dict(json_path)
total_queries = calculate_total_queries(anchor_query_dict, step_size)
print(f"Total number of query sequences after subsampling with step size {step_size}: {total_queries}")

total_anchors = calculate_total_anchors(anchor_query_dict, anchor_step_size)
print(f"Total number of anchor sequences after subsampling with anchor step size {anchor_step_size}: {total_anchors}")

Total number of query sequences after subsampling with step size 20: 412
Total number of anchor sequences after subsampling with anchor step size 10: 701


In [7]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
csv_file = "/home/bjangley/VPR/mast3r-v2/pairs_mining/spagna_train0/spagna_matches_inliers_fm_top3_anchors_per_query.csv"
df = pd.read_csv(csv_file)

# Filter the DataFrame to include only pairs with num_inliers > 200
filtered_df = df[df['num_inliers'] > 200]

# Count the number of rows in the filtered DataFrame
num_pairs = len(filtered_df)

# Print the result
print(f"Number of pairs with inliers > 200: {num_pairs}")

Number of pairs with inliers > 200: 767


In [9]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
csv_file = "/home/bjangley/VPR/mast3r-v2/pairs_mining/spagna_train0/spagna_matches_inliers_fm_top10_anchors_per_query.csv"
df = pd.read_csv(csv_file)

# Filter the DataFrame to include only pairs with num_inliers > 200
filtered_df = df[df['num_inliers'] > 200]

# Calculate the number of unique anchors in the filtered DataFrame
num_unique_anchors = filtered_df['anchor_idx'].nunique()

# Print the result
print(f"Number of pairs with inliers > 200: {len(filtered_df)}")
print(f"Number of unique anchors in the filtered dataset: {num_unique_anchors}")

Number of pairs with inliers > 200: 2485
Number of unique anchors in the filtered dataset: 174


In [11]:
import pandas as pd

def analyze_anchor_sets(train_file, test_file, val_file):
    """
    Analyzes train, test, and validation sets to determine:
    - The number of unique anchors in each set.
    - The size of the intersection between the sets.
    - The total number of pairs in each set.
    - The total number of unique anchors across all sets.

    Args:
        train_file (str): Path to the training set file.
        test_file (str): Path to the test set file.
        val_file (str): Path to the validation set file.

    Returns:
        None. Prints the analysis results.
    """

    def load_anchors(file_path):
        """Loads anchor and query indices from a file and returns them as sets."""
        try:
            df = pd.read_csv(file_path, sep=' ', header=None, names=['anchor_idx', 'query_idx'])
            anchor_set = set(df['anchor_idx'])
            num_pairs = len(df)
            return anchor_set, num_pairs
        except FileNotFoundError:
            print(f"Error: File not found at {file_path}")
            return set(), 0

    # Load anchor indices and number of pairs from each file
    train_anchors, num_train_pairs = load_anchors(train_file)
    test_anchors, num_test_pairs = load_anchors(test_file)
    val_anchors, num_val_pairs = load_anchors(val_file)

    # Calculate the number of unique anchors in each set
    num_train_anchors = len(train_anchors)
    num_test_anchors = len(test_anchors)
    num_val_anchors = len(val_anchors)

    # Calculate the intersection between the sets
    train_test_intersection = len(train_anchors.intersection(test_anchors))
    train_val_intersection = len(train_anchors.intersection(val_anchors))
    test_val_intersection = len(test_anchors.intersection(val_anchors))
    all_intersection = len(train_anchors.intersection(test_anchors).intersection(val_anchors))

    # Calculate the total number of unique anchors across all sets
    all_anchors = train_anchors.union(test_anchors).union(val_anchors)
    num_total_unique_anchors = len(all_anchors)

    # Print the results
    print("Number of pairs:")
    print(f"  Train: {num_train_pairs}")
    print(f"  Test: {num_test_pairs}")
    print(f"  Validation: {num_val_pairs}")

    print("\nNumber of unique anchors:")
    print(f"  Train: {num_train_anchors}")
    print(f"  Test: {num_test_anchors}")
    print(f"  Validation: {num_val_anchors}")
    print(f"  Total Unique Anchors: {num_total_unique_anchors}")

    print("\nIntersection sizes:")
    print(f"  Train and Test: {train_test_intersection}")
    print(f"  Train and Validation: {train_val_intersection}")
    print(f"  Test and Validation: {test_val_intersection}")
    print(f"  Train, Test, and Validation: {all_intersection}")



# Example usage:
train_file = "/home/bjangley/VPR/mast3r-v2/pairs_finetuning/spagna_train0/train_pairs.txt"  # Replace with the actual path to your train file
test_file = "/home/bjangley/VPR/mast3r-v2/pairs_finetuning/spagna_train0/test_pairs.txt"  # Replace with the actual path to your test file
val_file = "/home/bjangley/VPR/mast3r-v2/pairs_finetuning/spagna_train0/val_pairs.txt"  # Replace with the actual path to your validation file

analyze_anchor_sets(train_file, test_file, val_file)

Number of pairs:
  Train: 1739
  Test: 374
  Validation: 372

Number of unique anchors:
  Train: 172
  Test: 144
  Validation: 140
  Total Unique Anchors: 174

Intersection sizes:
  Train and Test: 144
  Train and Validation: 138
  Test and Validation: 121
  Train, Test, and Validation: 121


In [13]:
import pandas as pd

def analyze_anchor_sets(train_file, test_file, val_file):
    """
    Analyzes train, test, and validation sets to determine:
    - The number of unique anchors in each set.
    - The number of unique queries in each set.
    - The size of the intersection between the sets (anchor indices).
    - The size of the intersection between the sets (query indices).
    - The size of the intersection between the sets (anchor-query pairs).
    - The total number of pairs in each set.
    - The total number of unique anchors across all sets.
    - The total number of unique queries across all sets.

    Args:
        train_file (str): Path to the training set file.
        test_file (str): Path to the test set file.
        val_file (str): Path to the validation set file.

    Returns:
        None. Prints the analysis results.
    """

    def load_pairs(file_path):
        """Loads anchor and query indices from a file and returns them as sets."""
        try:
            df = pd.read_csv(file_path, sep=' ', header=None, names=['anchor_idx', 'query_idx'])
            # Create a set of (anchor_idx, query_idx) tuples
            pairs = set(zip(df['anchor_idx'], df['query_idx']))
            anchor_set = set(df['anchor_idx'])
            query_set = set(df['query_idx'])
            num_pairs = len(df)
            return anchor_set, query_set, pairs, num_pairs
        except FileNotFoundError:
            print(f"Error: File not found at {file_path}")
            return set(), set(), set(), 0

    # Load anchor and query indices and number of pairs from each file
    train_anchors, train_queries, train_pairs, num_train_pairs = load_pairs(train_file)
    test_anchors, test_queries, test_pairs, num_test_pairs = load_pairs(test_file)
    val_anchors, val_queries, val_pairs, num_val_pairs = load_pairs(val_file)

    # Calculate the number of unique anchors in each set
    num_train_anchors = len(train_anchors)
    num_test_anchors = len(test_anchors)
    num_val_anchors = len(val_anchors)

    # Calculate the number of unique queries in each set
    num_train_queries = len(train_queries)
    num_test_queries = len(test_queries)
    num_val_queries = len(val_queries)

    # Calculate the intersection between the sets (anchor indices)
    train_test_intersection_anchors = len(train_anchors.intersection(test_anchors))
    train_val_intersection_anchors = len(train_anchors.intersection(val_anchors))
    test_val_intersection_anchors = len(test_anchors.intersection(val_anchors))
    all_intersection_anchors = len(train_anchors.intersection(test_anchors).intersection(val_anchors))

    # Calculate the intersection between the sets (query indices)
    train_test_intersection_queries = len(train_queries.intersection(test_queries))
    train_val_intersection_queries = len(train_queries.intersection(val_queries))
    test_val_intersection_queries = len(test_queries.intersection(val_queries))
    all_intersection_queries = len(train_queries.intersection(test_queries).intersection(val_queries))

    # Calculate the intersection between the sets (anchor-query pairs)
    train_test_intersection_pairs = len(train_pairs.intersection(test_pairs))
    train_val_intersection_pairs = len(train_pairs.intersection(val_pairs))
    test_val_intersection_pairs = len(test_pairs.intersection(val_pairs))
    all_intersection_pairs = len(train_pairs.intersection(test_pairs).intersection(val_pairs))

    # Calculate the total number of unique anchors across all sets
    all_anchors = train_anchors.union(test_anchors).union(val_anchors)
    num_total_unique_anchors = len(all_anchors)

     # Calculate the total number of unique queries across all sets
    all_queries = train_queries.union(test_queries).union(val_queries)
    num_total_unique_queries = len(all_queries)

    # Print the results
    print("Number of pairs:")
    print(f"  Train: {num_train_pairs}")
    print(f"  Test: {num_test_pairs}")
    print(f"  Validation: {num_val_pairs}")

    print("\nNumber of unique anchors:")
    print(f"  Train: {num_train_anchors}")
    print(f"  Test: {num_test_anchors}")
    print(f"  Validation: {num_val_anchors}")
    print(f"  Total Unique Anchors: {num_total_unique_anchors}")

    print("\nNumber of unique queries:")
    print(f"  Train: {num_train_queries}")
    print(f"  Test: {num_test_queries}")
    print(f"  Validation: {num_val_queries}")
    print(f"  Total Unique Queries: {num_total_unique_queries}")

    print("\nIntersection sizes (anchor indices):")
    print(f"  Train and Test: {train_test_intersection_anchors}")
    print(f"  Train and Validation: {train_val_intersection_anchors}")
    print(f"  Test and Validation: {test_val_intersection_anchors}")
    print(f"  Train, Test, and Validation: {all_intersection_anchors}")

    print("\nIntersection sizes (query indices):")
    print(f"  Train and Test: {train_test_intersection_queries}")
    print(f"  Train and Validation: {train_val_intersection_queries}")
    print(f"  Test and Validation: {test_val_intersection_queries}")
    print(f"  Train, Test, and Validation: {all_intersection_queries}")

    print("\nIntersection sizes (anchor-query pairs):")
    print(f"  Train and Test: {train_test_intersection_pairs}")
    print(f"  Train and Validation: {train_val_intersection_pairs}")
    print(f"  Test and Validation: {test_val_intersection_pairs}")
    print(f"  Train, Test, and Validation: {all_intersection_pairs}")


# Example usage:
train_file = "/home/bjangley/VPR/mast3r-v2/pairs_finetuning/spagna_train0/train_pairs.txt"  # Replace with the actual path to your train file
test_file = "/home/bjangley/VPR/mast3r-v2/pairs_finetuning/spagna_train0/test_pairs.txt"  # Replace with the actual path to your test file
val_file = "/home/bjangley/VPR/mast3r-v2/pairs_finetuning/spagna_train0/val_pairs.txt"  # Replace with the actual path to your validation file

analyze_anchor_sets(train_file, test_file, val_file)

Number of pairs:
  Train: 1739
  Test: 374
  Validation: 372

Number of unique anchors:
  Train: 172
  Test: 144
  Validation: 140
  Total Unique Anchors: 174

Number of unique queries:
  Train: 256
  Test: 194
  Validation: 200
  Total Unique Queries: 257

Intersection sizes (anchor indices):
  Train and Test: 144
  Train and Validation: 138
  Test and Validation: 121
  Train, Test, and Validation: 121

Intersection sizes (query indices):
  Train and Test: 193
  Train and Validation: 199
  Test and Validation: 146
  Train, Test, and Validation: 145

Intersection sizes (anchor-query pairs):
  Train and Test: 0
  Train and Validation: 0
  Test and Validation: 0
  Train, Test, and Validation: 0
